# 1个卷积层（One Convolution Layer）

In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 30 20:11:19 2016

@author: stephen
"""
 
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
    x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
    
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))

    
    x = keras.layers.Input(x_train.shape[1:])
#    drop_out = Dropout(0.2)(x)
    conv1 = keras.layers.Conv2D(128, 5, 1, border_mode='same')(x)
    conv1 = keras.layers.normalization.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation('relu')(conv1)
    
    full = keras.layers.pooling.GlobalAveragePooling2D()(conv1)    
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    
    model = Model(input=x, output=out)
     
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 0s - loss: 3.6802 - acc: 0.0205 - val_loss: 3.6151 - val_acc: 0.0205
Epoch 2/2000
390/390 [==============================] - 0s - loss: 3.6144 - acc: 0.0308 - val_loss: 3.6192 - val_acc: 0.0179
Epoch 3/2000
390/390 [==============================] - 0s - loss: 3.6074 - acc: 0.0308 - val_loss: 3.6213 - val_acc: 0.0179
Epoch 4/2000
390/390 [==============================] - 0s - loss: 3.6043 - acc: 0.0231 - val_loss: 3.6224 - val_acc: 0.0179
Epoch 5/2000
390/390 [==============================] - 0s - loss: 3.6020 - acc: 0.0308 - val_loss: 3.6240 - val_acc: 0.0179
Epoch 6/2000
390/390 [==============================] - 0s - loss: 3.6024 - acc: 0.0359 - val_loss: 3.6278 - val_acc: 0.0179
Epoch 7/2000
390/390 [==============================] - 0s - loss: 3.6036 - acc: 0.0256 - val_loss: 3.6271 - val_acc: 0.0179
Epoch 8/2000
390/390 [==============================] - 0s - loss: 3.6000 - acc

390/390 [==============================] - 0s - loss: 3.5948 - acc: 0.0410 - val_loss: 3.6962 - val_acc: 0.0230
Epoch 66/2000
390/390 [==============================] - 0s - loss: 3.6007 - acc: 0.0487 - val_loss: 3.6893 - val_acc: 0.0384
Epoch 67/2000
390/390 [==============================] - 0s - loss: 3.5897 - acc: 0.0308 - val_loss: 3.6753 - val_acc: 0.0205
Epoch 68/2000
390/390 [==============================] - 0s - loss: 3.5910 - acc: 0.0385 - val_loss: 3.6788 - val_acc: 0.0307
Epoch 69/2000
390/390 [==============================] - 0s - loss: 3.5949 - acc: 0.0359 - val_loss: 3.6886 - val_acc: 0.0230
Epoch 70/2000
390/390 [==============================] - 0s - loss: 3.5923 - acc: 0.0385 - val_loss: 3.6839 - val_acc: 0.0205
Epoch 71/2000
390/390 [==============================] - 0s - loss: 3.5927 - acc: 0.0333 - val_loss: 3.6889 - val_acc: 0.0179
Epoch 72/2000
390/390 [==============================] - 0s - loss: 3.5898 - acc: 0.0256 - val_loss: 3.6772 - val_acc: 0.0205
Epoch 

390/390 [==============================] - 1s - loss: 3.5846 - acc: 0.0385 - val_loss: 3.6699 - val_acc: 0.0460
Epoch 130/2000
390/390 [==============================] - 1s - loss: 3.5822 - acc: 0.0487 - val_loss: 3.6686 - val_acc: 0.0230
Epoch 131/2000
390/390 [==============================] - 1s - loss: 3.5861 - acc: 0.0436 - val_loss: 3.6906 - val_acc: 0.0256
Epoch 132/2000
390/390 [==============================] - 1s - loss: 3.5837 - acc: 0.0513 - val_loss: 3.6704 - val_acc: 0.0614
Epoch 133/2000
390/390 [==============================] - 1s - loss: 3.5785 - acc: 0.0513 - val_loss: 3.6697 - val_acc: 0.0230
Epoch 134/2000
390/390 [==============================] - 1s - loss: 3.5852 - acc: 0.0333 - val_loss: 3.6686 - val_acc: 0.0230
Epoch 135/2000
390/390 [==============================] - 1s - loss: 3.5835 - acc: 0.0333 - val_loss: 3.6793 - val_acc: 0.0435
Epoch 136/2000
390/390 [==============================] - 1s - loss: 3.5832 - acc: 0.0462 - val_loss: 3.6765 - val_acc: 0.0588

390/390 [==============================] - 1s - loss: 3.5666 - acc: 0.0538 - val_loss: 3.6658 - val_acc: 0.0281
Epoch 194/2000
390/390 [==============================] - 1s - loss: 3.5714 - acc: 0.0513 - val_loss: 3.6557 - val_acc: 0.0435
Epoch 195/2000
390/390 [==============================] - 1s - loss: 3.5665 - acc: 0.0359 - val_loss: 3.6580 - val_acc: 0.0230
Epoch 196/2000
390/390 [==============================] - 1s - loss: 3.5664 - acc: 0.0615 - val_loss: 3.6650 - val_acc: 0.0256
Epoch 197/2000
390/390 [==============================] - 1s - loss: 3.5681 - acc: 0.0538 - val_loss: 3.6543 - val_acc: 0.0870
Epoch 198/2000
390/390 [==============================] - 1s - loss: 3.5640 - acc: 0.0615 - val_loss: 3.6626 - val_acc: 0.0256
Epoch 199/2000
390/390 [==============================] - 1s - loss: 3.5673 - acc: 0.0667 - val_loss: 3.6623 - val_acc: 0.0537
Epoch 200/2000
390/390 [==============================] - 1s - loss: 3.5687 - acc: 0.0590 - val_loss: 3.6666 - val_acc: 0.0281

...

390/390 [==============================] - 1s - loss: 2.2831 - acc: 0.4179 - val_loss: 2.6062 - val_acc: 0.2225
Epoch 1729/2000
390/390 [==============================] - 1s - loss: 2.3006 - acc: 0.4256 - val_loss: 2.4080 - val_acc: 0.2890
Epoch 1730/2000
390/390 [==============================] - 1s - loss: 2.3076 - acc: 0.3718 - val_loss: 2.5226 - val_acc: 0.2609
Epoch 1731/2000
390/390 [==============================] - 1s - loss: 2.3021 - acc: 0.3718 - val_loss: 2.4682 - val_acc: 0.2685
Epoch 1732/2000
390/390 [==============================] - 1s - loss: 2.3072 - acc: 0.3487 - val_loss: 2.7490 - val_acc: 0.1739
Epoch 1733/2000
390/390 [==============================] - 1s - loss: 2.2739 - acc: 0.4077 - val_loss: 2.9726 - val_acc: 0.1355
Epoch 1734/2000
390/390 [==============================] - 1s - loss: 2.3007 - acc: 0.4385 - val_loss: 3.3143 - val_acc: 0.1176
Epoch 1735/2000
390/390 [==============================] - 1s - loss: 2.2919 - acc: 0.3846 - val_loss: 2.8362 - val_acc:

390/390 [==============================] - 1s - loss: 2.2694 - acc: 0.4103 - val_loss: 2.4944 - val_acc: 0.3146
Epoch 1793/2000
390/390 [==============================] - 2s - loss: 2.2676 - acc: 0.4103 - val_loss: 2.8668 - val_acc: 0.1509
Epoch 1794/2000
390/390 [==============================] - 1s - loss: 2.2662 - acc: 0.3718 - val_loss: 2.3873 - val_acc: 0.2788
Epoch 1795/2000
390/390 [==============================] - 1s - loss: 2.2611 - acc: 0.4205 - val_loss: 2.4734 - val_acc: 0.3095
Epoch 1796/2000
390/390 [==============================] - 1s - loss: 2.2525 - acc: 0.4231 - val_loss: 2.4071 - val_acc: 0.3095
Epoch 1797/2000
390/390 [==============================] - 1s - loss: 2.2406 - acc: 0.4385 - val_loss: 2.3653 - val_acc: 0.3223
Epoch 1798/2000
390/390 [==============================] - 1s - loss: 2.2350 - acc: 0.4308 - val_loss: 2.3855 - val_acc: 0.3274
Epoch 1799/2000
390/390 [==============================] - 1s - loss: 2.2391 - acc: 0.4154 - val_loss: 2.3433 - val_acc:

390/390 [==============================] - 1s - loss: 2.2233 - acc: 0.4462 - val_loss: 2.5953 - val_acc: 0.2123
Epoch 1857/2000
390/390 [==============================] - 1s - loss: 2.2364 - acc: 0.4282 - val_loss: 2.4109 - val_acc: 0.2583
Epoch 1858/2000
390/390 [==============================] - 1s - loss: 2.2106 - acc: 0.4538 - val_loss: 2.3781 - val_acc: 0.3274
Epoch 1859/2000
390/390 [==============================] - 1s - loss: 2.2475 - acc: 0.4000 - val_loss: 2.3545 - val_acc: 0.3248
Epoch 1860/2000
390/390 [==============================] - 1s - loss: 2.2368 - acc: 0.3949 - val_loss: 2.3125 - val_acc: 0.3453
Epoch 1861/2000
390/390 [==============================] - 1s - loss: 2.2212 - acc: 0.4128 - val_loss: 2.5763 - val_acc: 0.2046
Epoch 1862/2000
390/390 [==============================] - 1s - loss: 2.2338 - acc: 0.3923 - val_loss: 2.7312 - val_acc: 0.1918
Epoch 1863/2000
390/390 [==============================] - 1s - loss: 2.2117 - acc: 0.4128 - val_loss: 2.3308 - val_acc:

390/390 [==============================] - 1s - loss: 2.2315 - acc: 0.3897 - val_loss: 2.5590 - val_acc: 0.2839
Epoch 1921/2000
390/390 [==============================] - 1s - loss: 2.2163 - acc: 0.4000 - val_loss: 2.4299 - val_acc: 0.2379
Epoch 1922/2000
390/390 [==============================] - 1s - loss: 2.2231 - acc: 0.4077 - val_loss: 2.3378 - val_acc: 0.3043
Epoch 1923/2000
390/390 [==============================] - 1s - loss: 2.2311 - acc: 0.4026 - val_loss: 2.4288 - val_acc: 0.2788
Epoch 1924/2000
390/390 [==============================] - 1s - loss: 2.2196 - acc: 0.4179 - val_loss: 2.5324 - val_acc: 0.2123
Epoch 1925/2000
390/390 [==============================] - 1s - loss: 2.2028 - acc: 0.3923 - val_loss: 2.6568 - val_acc: 0.2634
Epoch 1926/2000
390/390 [==============================] - 1s - loss: 2.1912 - acc: 0.4103 - val_loss: 2.4411 - val_acc: 0.2890
Epoch 1927/2000
390/390 [==============================] - 1s - loss: 2.2065 - acc: 0.4179 - val_loss: 2.3768 - val_acc:

390/390 [==============================] - 1s - loss: 2.1886 - acc: 0.3846 - val_loss: 2.8395 - val_acc: 0.1841
Epoch 1985/2000
390/390 [==============================] - 1s - loss: 2.1706 - acc: 0.3974 - val_loss: 2.5852 - val_acc: 0.2020
Epoch 1986/2000
390/390 [==============================] - 1s - loss: 2.1748 - acc: 0.4410 - val_loss: 2.3244 - val_acc: 0.3350
Epoch 1987/2000
390/390 [==============================] - 1s - loss: 2.1740 - acc: 0.4333 - val_loss: 2.2663 - val_acc: 0.3939
Epoch 1988/2000
390/390 [==============================] - 2s - loss: 2.1748 - acc: 0.4564 - val_loss: 2.2645 - val_acc: 0.3632
Epoch 1989/2000
390/390 [==============================] - 1s - loss: 2.1682 - acc: 0.4359 - val_loss: 2.2671 - val_acc: 0.3350
Epoch 1990/2000
390/390 [==============================] - 1s - loss: 2.2164 - acc: 0.3923 - val_loss: 2.3723 - val_acc: 0.3043
Epoch 1991/2000
390/390 [==============================] - 1s - loss: 2.1673 - acc: 0.4308 - val_loss: 2.3789 - val_acc: